In [1]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [2]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+6]
        df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, error_method='Cramer', error_label=' Error (SD)', output='both'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise KeyError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    down_amp = df[down].astype(float).sum(axis=1)
    if error_method.lower() == 'cramer':
        up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
        down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    elif error_method.lower()=='narrow':
        up_err = df[up_err_name].astype(float).max(axis=1) 
        down_err = df[down_err_name].astype(float).max(axis=1) 
    else:
        raise KeyError('Please choose "Cramer" or "Narrow" for the error_method')
    
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label =f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD) {error_method}'
    
    if output=='both':
        return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})
    elif output=='ratio':
        return pd.DataFrame({col_name: ratio})      
    elif output=='error':
        return pd.DataFrame({col_error_name: ratio_err})   
    else:
        raise KeyError('Please choose "ratio", "error" or "both" (default) as output')

In [3]:
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
data_folder = Path('/Users/patxi/Sync/MRdata/IoN_Piglet/Ellie')
data_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')
azi_data = Path('/Users/patxi/Desktop/AZI_MRS')
Ray_phd = Path('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs')
Ray_phd = Path('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs')
MELEth = Path('/Users/patxi/Sync/MRdata/IoN_Piglet/For_RAY_MELEth_22Dec2022')

missing_INSPIRE = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY')

data_folder = missing_INSPIRE

basis_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "csi" in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_both.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    # print(csv)
    if not csv.is_file():
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        command = f'tarquin --rows 3 --cols 2 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        print(f'{command}\n') 
        os.system(command)


  0%|          | 0/9 [00:00<?, ?it/s]

tarquin --rows 3 --cols 2 --input /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_CSI_PB_auto_TR2S_SENSE_10_2_raw_act.SDAT --output_csv /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_CSI_PB_auto_TR2S_SENSE_10_2_raw_act_both.csv --input_w /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_CSI_PB_auto_TR2S_SENSE_10_2_raw_ref.SDAT --basis_csv /Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM


2023-Jan-03 09:44:10 INFO   : TARQUIN 4.3.10 Started
2023-Jan-03 09:44:10 INFO   : Loading data file : /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_CSI_PB_auto_TR2S_SENSE_10_2_raw_act.SDAT
2023-Jan-03 09:44:10 INFO   : Loading data file : /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_CSI_PB_auto_TR2S_SENSE_10_2_raw_act.SDAT
2023-Jan-03 09:44:10 INFO   : nSt

.............


2023-Jan-03 09:44:18 INFO   : done.
2023-Jan-03 09:44:18 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM/Glc.csv'

.........................

...


2023-Jan-03 09:46:14 INFO   : done.
2023-Jan-03 09:46:14 DEBUG  : Final Beta: 0.00
2023-Jan-03 09:46:14 DEBUG  : Final Phi0: 0.22
2023-Jan-03 09:46:14 DEBUG  : Final Phi1: 0.00
2023-Jan-03 09:46:14 INFO   : FWHM right ppm = 3.200143
2023-Jan-03 09:46:14 INFO   : FWHM left ppm = 3.228796
2023-Jan-03 09:46:14 INFO   : Metabolite FWHM (ppm) = 0.028653
2023-Jan-03 09:46:14 INFO   : Spec noise            = 0.010060
2023-Jan-03 09:46:14 INFO   : sdev noise = 0.591582
2023-Jan-03 09:46:14 INFO   : SNR residual = 3.381564
2023-Jan-03 09:46:14 INFO   : SNR max      = 4.394192
2023-Jan-03 09:46:14 INFO   : Ymax       = 4.000946
2023-Jan-03 09:46:14 INFO   : Ymax metab = 4.000946
2023-Jan-03 09:46:14 INFO   : Fit quality = 1.299455
2023-Jan-03 09:46:14 INFO   : Baseline dev = 0.027109
2023-Jan-03 09:46:14 INFO   : Ymax         = 4.000946
2023-Jan-03 09:46:14 INFO   : BSL         = -0.034635
2023-Jan-03 09:46:14 INFO   : Computing CRLBs.
2023-Jan-03 09:46:14 DEBUG  : Sigma: 1.075468e-04
2023-Jan-

...........................................................................................................................................................................................................................................................


2023-Jan-03 09:46:16 INFO   : done.
2023-Jan-03 09:46:16 DEBUG  : Final Beta: 4.59
2023-Jan-03 09:46:16 DEBUG  : Final Phi0: 0.60
2023-Jan-03 09:46:16 DEBUG  : Final Phi1: -0.00
2023-Jan-03 09:46:16 INFO   : FWHM right ppm = 2.000525
2023-Jan-03 09:46:16 INFO   : FWHM left ppm = 2.025357
2023-Jan-03 09:46:16 INFO   : Metabolite FWHM (ppm) = 0.024833
2023-Jan-03 09:46:16 INFO   : Spec noise            = 0.010099
2023-Jan-03 09:46:16 INFO   : sdev noise = 0.603957
2023-Jan-03 09:46:16 INFO   : SNR residual = 7.747874
2023-Jan-03 09:46:16 INFO   : SNR max      = 10.238480
2023-Jan-03 09:46:16 INFO   : Ymax       = 9.358759
2023-Jan-03 09:46:16 INFO   : Ymax metab = 9.358759
2023-Jan-03 09:46:16 INFO   : Fit quality = 1.321457
2023-Jan-03 09:46:16 INFO   : Baseline dev = 0.014760
2023-Jan-03 09:46:16 INFO   : Ymax         = 9.358759
2023-Jan-03 09:46:16 INFO   : BSL         = -0.023471
2023-Jan-03 09:46:16 INFO   : Computing CRLBs.
2023-Jan-03 09:46:16 DEBUG  : Sigma: 1.539407e-04
2023-Ja

100%|██████████| 9/9 [02:06<00:00, 14.04s/it]........................................................................................................................


2023-Jan-03 09:46:17 INFO   : done.
2023-Jan-03 09:46:17 DEBUG  : Final Beta: 42.77
2023-Jan-03 09:46:17 DEBUG  : Final Phi0: 0.36
2023-Jan-03 09:46:17 DEBUG  : Final Phi1: -0.00
2023-Jan-03 09:46:17 INFO   : FWHM right ppm = 2.003390
2023-Jan-03 09:46:17 INFO   : FWHM left ppm = 2.037774
2023-Jan-03 09:46:17 INFO   : Metabolite FWHM (ppm) = 0.034384
2023-Jan-03 09:46:17 INFO   : Spec noise            = 0.011192
2023-Jan-03 09:46:17 INFO   : sdev noise = 0.621637
2023-Jan-03 09:46:17 INFO   : SNR residual = 7.402481
2023-Jan-03 09:46:17 INFO   : SNR max      = 9.085272
2023-Jan-03 09:46:17 INFO   : Ymax       = 9.203317
2023-Jan-03 09:46:17 INFO   : Ymax metab = 9.203317
2023-Jan-03 09:46:17 INFO   : Fit quality = 1.227328
2023-Jan-03 09:46:17 INFO   : Baseline dev = 0.012850
2023-Jan-03 09:46:17 INFO   : Ymax         = 9.203317
2023-Jan-03 09:46:17 INFO   : BSL         = -0.011997
2023-Jan-03 09:46:17 INFO   : Computing CRLBs.
2023-Jan-03 09:46:17 DEBUG  : Sigma: 1.341222e-04
2023-Ja

In [4]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act_both.csv" in f.name.lower() and "csi" in f.name.lower())]



rows_to_process = {'BGT': 5, #(BGT = Basal Ganglia and Thalamic, ie deep GM)
                   'WM': 3,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        data_list.append([*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+8),*pull_from_csv(file,header=16,row=row_num+1)])
    my_columns = list(pd.read_csv(file, header=[1], error_bad_lines=False))
    my_errors = [item+' Error (SD)' for item in my_columns]
    my_diagnostics = pull_from_csv(file,row=0, header=16)    
    df = pd.DataFrame(data_list, columns=['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ])
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    df_L2N_narrow_error = met_ratio(df,['Lac', 'Threonine'], 'TNAA', error_method='Narrow', output='error')
    df_N2C = met_ratio(df,'TNAA', 'TCho')
    df_C2C = met_ratio(df,'TCho', 'Cr')
    df_N2Cr = met_ratio(df,'TNAA', 'Cr')


    # df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    df_ratio = pd.concat((df_L2N,df_L2N_narrow_error, df_N2C, df_C2C, df_N2Cr, df), axis=1)

    # df_ratio.set_index('LWP').to_excel(f'{label}_RATIOS_INSPIRE.xlsx')
    # df_ratio.set_index('LWP').to_excel(f'{label}_MELEth_PLOTS.xlsx')
    df_ratio.set_index('LWP').to_excel(f'{label}_MISSING_INSPIRE.xlsx')




/var/folders/qr/hf3x765n1mjgzkb2_by7zrcc0000gx/T/ipykernel_59015/710225691.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
b'Skipping line 18: expected 25 fields, saw 33\nSkipping line 19: expected 25 fields, saw 33\nSkipping line 20: expected 25 fields, saw 33\nSkipping line 21: expected 25 fields, saw 33\nSkipping line 22: expected 25 fields, saw 33\nSkipping line 23: expected 25 fields, saw 33\nSkipping line 24: expected 25 fields, saw 33\n'
/var/folders/qr/hf3x765n1mjgzkb2_by7zrcc0000gx/T/ipykernel_59015/710225691.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
b'Skipping line 18: expected 25 fields, saw 33\nSkipping line 19: expected 25 fields, saw 33\nSk

In [5]:
dfbb = met_ratio(df,'TNAA', 'TCho', error_method='Narrow', output='error')
dfbb


,TNAA/TCho Error (SD) Narrow
0,0.350245
1,0.498699
2,0.638957
3,0.260260
4,0.727230
5,0.637768
6,0.416988
7,0.764880
